### In this notebook we investigate a designed simple Inception network on PDU data

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

### Importing the libraries

In [2]:
import torch 

import torch.nn as nn
import torch.utils.data as Data
from torch.autograd import Function, Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn

from pathlib import Path
import os
import copy
import math
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime
import time as time

import warnings

#### Checking whether the GPU is active

In [3]:
torch.backends.cudnn.enabled

True

In [4]:
torch.cuda.is_available()

True

In [5]:
torch.cuda.init()

#### Dataset paths

In [6]:
PATH = Path("/home/saman/Saman/data/PDU_Raw_Data01/Test06_600x30/")
train_path = PATH / 'train' / 'Total'
valid_path = PATH / 'valid' / 'Total'
test_path = PATH / 'test' / 'Total'

### Model parameters

In [7]:
Num_Filter1= 16
Num_Filter2= 64
Ker_Sz1 = 5
Ker_Sz2 = 5

learning_rate= 0.0001

Dropout= 0.2
BchSz= 32
EPOCH= 5

### Data Augmenation

In [8]:
# Mode of transformation
transformation = transforms.Compose([
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0,0,0), (0.5,0.5,0.5)),
]) 

transformation2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0,0,0), (0.5,0.5,0.5)),    
]) 

In [9]:
# Loss calculator
criterion = nn.CrossEntropyLoss()   # cross entropy loss

### Defining models

#### Defining a class of our simple model

In [10]:
class ConvNet(nn.Module):
    def __init__(self,  Num_Filter1 , Num_Filter2, Ker_Sz1, Ker_Sz2,  Dropout, num_classes=2):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(  
            nn.Conv2d(              # input shape (3, 30, 600)
                in_channels=3,      # input height
                out_channels=Num_Filter1,    # n_filters
                kernel_size=Ker_Sz1,      # Kernel size
                stride=1,           # filter movement/step
                padding=int((Ker_Sz1-1)/2), # if want same width and length of this image after con2d,
            ),                              # padding=(kernel_size-1)/2 if stride=1
            nn.BatchNorm2d(Num_Filter1),     # Batch Normalization
            nn.ReLU(),              # Rectified linear activation
            nn.MaxPool2d(kernel_size=2, stride=2)) # choose max value in 2x2 area, 
                                                   
        # Visualizing this in https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(Num_Filter1, Num_Filter2, 
                      kernel_size=Ker_Sz2, 
                      stride=1, 
                      padding=int((Ker_Sz2-1)/2)),
            nn.BatchNorm2d(Num_Filter2),                              
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # output shape (64, 38, 38)
            nn.Dropout2d(p=Dropout))
        
        self.fc = nn.Linear(1050*Num_Filter2, num_classes) # fully connected layer, output 2 classes

        
        
    def forward(self, x):                  # Forwarding the data to classifier 
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1) # flatten the output of conv2 to (batch_size, 64*38*38)
        out = self.fc(out)
        return out

### Defining inception classes

In [11]:
class BasicConv2d(nn.Module):

    def __init__(self, in_planes, out_planes, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_planes, out_planes, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_planes, eps=0.001)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        out = self.relu(x)
        return x

In [12]:
class Inception(nn.Module):

    def __init__(self, in_channels):
        super(Inception, self).__init__()
        self.branch3x3 = BasicConv2d(in_channels, 384, kernel_size=3, stride=2)

        self.branch3x3dbl_1 = BasicConv2d(in_channels, 64, kernel_size=1)
        self.branch3x3dbl_2 = BasicConv2d(64, 96, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = BasicConv2d(96, 96, kernel_size=3, stride=2)
        
        self.avgpool = nn.AvgPool2d(kernel_size=3, stride=2)

    def forward(self, x):
        branch3x3 = self.branch3x3(x)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        branch_pool = self.avgpool(x)

        outputs = [branch3x3, branch3x3dbl, branch_pool]
        return torch.cat(outputs, 1)

In [13]:
class Inception_Net(nn.Module):
    def __init__(self,  Num_Filter1 , Num_Filter2, Ker_Sz1, Ker_Sz2,  Dropout, num_classes=2):
        super(Inception_Net, self).__init__()
        self.layer1 = nn.Sequential(  
            nn.Conv2d(              # input shape (3, 30, 600)
                in_channels=3,      # input height
                out_channels=Num_Filter1,    # n_filters
                kernel_size=Ker_Sz1,      # Kernel size
                stride=1,           # filter movement/step
                padding=int((Ker_Sz1-1)/2), # if want same width and length of this image after con2d,
            ),                              # padding=(kernel_size-1)/2 if stride=1
            nn.BatchNorm2d(Num_Filter1),     # Batch Normalization
            nn.ReLU(),              # Rectified linear activation
            nn.MaxPool2d(kernel_size=2, stride=2)) # choose max value in 2x2 area, 
                                                   
        # Visualizing this in https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(Num_Filter1, Num_Filter2, 
                      kernel_size=Ker_Sz2, 
                      stride=1, 
                      padding=int((Ker_Sz2-1)/2)),
            nn.BatchNorm2d(Num_Filter2),                              
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # output shape (64, 38, 38)
            nn.Dropout2d(p=Dropout))
        
        self.Inception = Inception(Num_Filter2)
            
        self.fc = nn.Linear(120768, num_classes) # fully connected layer, output 2 classes
        
    def forward(self, x):                  # Forwarding the data to classifier 
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.Inception(out)
        out = out.reshape(out.size(0), -1) # flatten the output of conv2 to (batch_size, 64*38*38)
        out = self.fc(out)
        return out

### Finding number of parameter in our model

In [14]:
def print_num_params(model):
    TotalParam=0
    for param in list(model.parameters()):
        print("Individual parameters are:")
        nn=1
        for size in list(param.size()):
            print(size)
            nn = nn*size
        print("Total parameters: {}" .format(param.numel()))
        TotalParam += nn
    print('-' * 10)
    print("Sum of all Parameters is: {}" .format(TotalParam))

In [15]:
def get_num_params(model):
    TotalParam=0
    for param in list(model.parameters()):
        nn=1
        for size in list(param.size()):
            nn = nn*size
        TotalParam += nn
    return TotalParam

### Training and Validating

#### Training and validation function

In [16]:
def train_model(model, criterion, optimizer,  Dropout, learning_rate,  BATCHSIZE, num_epochs):
        print(str(datetime.now()).split('.')[0], "Starting training and validation...\n")
        print("====================Data and Hyperparameter Overview====================\n")
        print("Number of training examples: {} , Number of validation examples: {} \n".format(len(train_data), len(valid_data)))
              
        print("Dropout:{:,.2f}, Learning rate: {:,.5f} " 
              .format( Dropout, learning_rate ))        
        print("Batch size: {}, Number of epochs: {} " 
              .format(BATCHSIZE, num_epochs)) 
        
        print("Number of parameter in the model: {}". format(get_num_params(model)))
              
        print("================================Results...==============================\n")

        since = time.time()  #record the beginning time

        best_model = model
        best_acc = 0.0
        acc_vect =[]   

        for epoch in range(num_epochs):
            for i, (images, labels) in enumerate(train_loader):   
                images = Variable(images).cuda()
                labels = Variable(labels).cuda()

                # Forward pass
                outputs = model(images)            # model output
                loss = criterion(outputs, labels)  # cross entropy loss

                # Trying binary cross entropy
                #loss = criterion(torch.max(outputs.data, 1), labels)
                #loss = torch.nn.functional.binary_cross_entropy(outputs, labels)
                
                

                # Backward and optimize
                optimizer.zero_grad()             # clear gradients for this training step
                loss.backward()                   # backpropagation, compute gradients
                optimizer.step()                  # apply gradients

                if (i+1) % 1000 == 0:               # Reporting the loss and progress every 50 step
                    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                               .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

            model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)

            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in valid_loader:
                    images = Variable(images).cuda()
                    labels = Variable(labels).cuda()
                    
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)

                    loss = criterion(outputs, labels)
                    loss += loss.item()

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

                epoch_loss= loss / total
                epoch_acc = 100 * correct / total
                acc_vect.append(epoch_acc)

                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model = copy.deepcopy(model)

                print('Validation accuracy and loss of the model on  {} images: {} %, {:.5f}'
                      .format(len(valid_data), 100 * correct / total, loss))

            correct = 0
            total = 0
            for images, labels in train_loader:
                images = Variable(images).cuda()
                labels = Variable(labels).cuda()
                
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)

                loss = criterion(outputs, labels)
                loss += loss.item()

                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            epoch_loss= loss / total
            epoch_acc = 100 * correct / total

            print('Train  accuracy and loss of the model on  {} images: {} %, {:.5f}'
                  .format(len(train_data), epoch_acc, loss))
            print('-' * 10)

        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
        print('Best validation Acc: {:4f}'.format(best_acc)) 
        
        mean_acc = np.mean(acc_vect)
        print('Average accuracy on the validation {} images: {}'
              .format(len(train_data),mean_acc))
        print('-' * 10)
        return best_model, mean_acc

### Testing function

In [17]:
def test_model(model, test_loader):
    print("Starting testing...\n")
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)

    with torch.no_grad():
        correct = 0
        total = 0
        test_loss_vect=[]
        test_acc_vect=[]
        
        since = time.time()  #record the beginning time
        
        for i in range(10):
            
            Indx = torch.randperm(len(test_data))
            Cut=int(len(Indx)/10) # Here 10% showing the proportion of data is chosen for pooling
            indices=Indx[:Cut]            
            Sampler = Data.SubsetRandomSampler(indices)
            pooled_data =  torch.utils.data.DataLoader(test_data , batch_size=BchSz,sampler=Sampler)

            for images, labels in pooled_data:
                images = Variable(images).cuda()
                labels = Variable(labels).cuda()
                
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
            test_loss= loss / total
            test_accuracy= 100 * correct / total
            
            test_loss_vect.append(test_loss)
            test_acc_vect.append(test_accuracy)

            
#             print('Test accuracy and loss for the {}th pool: {:.2f} %, {:.5f}'
#                   .format(i+1, test_accuracy, test_loss))
            
        
        mean_test_loss = np.mean(test_loss_vect)
        mean_test_acc = np.mean(test_acc_vect)
        std_test_acc = np.std(test_acc_vect)
        
        print('-' * 10)
        print('Average test accuracy on test data: {:.2f} %, loss: {:.5f}, Standard deviion of accuracy: {:.4f}'
              .format(mean_test_acc, mean_test_loss, std_test_acc))
        
        print('-' * 10)
        time_elapsed = time.time() - since
        print('Testing complete in {:.1f}m {:.4f}s'.format(time_elapsed // 60, time_elapsed % 60))
        
        print('-' * 10)
        
        return mean_test_acc, mean_test_loss, std_test_acc

### Applying aumentation and batch size

In [18]:
## Using batch size to load data
train_data = torchvision.datasets.ImageFolder(train_path,transform=transformation)
train_loader =torch.utils.data.DataLoader(train_data, batch_size=BchSz, shuffle=True,
                                          num_workers=8)

valid_data = torchvision.datasets.ImageFolder(valid_path,transform=transformation)
valid_loader =torch.utils.data.DataLoader(valid_data, batch_size=BchSz, shuffle=True,
                                          num_workers=8)

test_data = torchvision.datasets.ImageFolder(test_path,transform=transformation2)
test_loader =torch.utils.data.DataLoader(test_data, batch_size=BchSz, shuffle=True,
                                          num_workers=8)

In [19]:
model = Inception_Net(Num_Filter1 , Num_Filter2, Ker_Sz1, Ker_Sz2, Dropout, num_classes=2)

model = model.cuda()
print(model)

# Defining optimizer with variable learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer.scheduler=lr_scheduler.ReduceLROnPlateau(optimizer, 'min')  

Inception_Net(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout2d(p=0.2)
  )
  (Inception): Inception(
    (branch3x3): BasicConv2d(
      (conv): Conv2d(64, 384, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(384, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
    )
    (branch3x3dbl_1): BasicConv2d(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     

In [20]:
get_num_params(model)

633378

In [20]:
seed= [1, 3, 7, 19, 22]

val_acc_vect=[]
test_acc_vect=[]


for ii in seed: 
    torch.cuda.manual_seed(ii)
    torch.manual_seed(ii)
    
    model, val_acc= train_model(model, criterion, optimizer,  Dropout, learning_rate,  BchSz, EPOCH)
    testing = test_model (model, test_loader)
    test_acc= testing[0]
    
    
    val_acc_vect.append( val_acc )
    test_acc_vect.append(test_acc)
    
    mean_val_acc = np.mean(val_acc_vect)
    mean_test_acc = np.mean(test_acc_vect)
    
    
print('-' * 10)
print('-' * 10)
print('Average of validation accuracies on 5 different random seed: {:.2f} %, Average of testing accuracies on 5 different random seed: {:.2f} %'
      .format(mean_val_acc, mean_test_acc))    
    

2019-03-01 15:11:27 Starting training and validation...

====================Data and Hyperparameter Overview====================

Number of training examples: 24000 , Number of validation examples: 8000 

Dropout:0.20, Learning rate: 0.00010 
Batch size: 32, Number of epochs: 5 
Number of parameter in the model: 633378
================================Results...==============================

Validation accuracy and loss of the model on  8000 images: 64.9 %, 1.33086
Train  accuracy and loss of the model on  24000 images: 62.7375 %, 1.01242
----------
Validation accuracy and loss of the model on  8000 images: 75.4 %, 0.76369
Train  accuracy and loss of the model on  24000 images: 77.225 %, 1.38264
----------
Validation accuracy and loss of the model on  8000 images: 77.35 %, 1.22606
Train  accuracy and loss of the model on  24000 images: 87.25833333333334 %, 0.64452
----------
Validation accuracy and loss of the model on  8000 images: 72.8875 %, 0.65668
Train  accuracy and loss of the m